In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

### ● 不動産の一つの物件から情報を取得する。

homes、新宿区のみで実施

In [2]:
base_url = "https://www.homes.co.jp/chintai/tokyo/shinjuku-city/list/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(base_url, headers=headers)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="mod-mergeBuilding--rent--photo rMansion ui-frame ui-frame-cacao-bar")

In [3]:

property_name = items.find(class_="bukkenName").get_text(strip=True) if items.find(class_="bukkenName") else None
    
for item in items:
    address_td = item.find('th', string='所在地')
    if address_td:
        address = address_td.find_next_sibling('td').get_text(strip=True)
        break

nearest_stations = []
traffic_td = items.find("td", class_="traffic")
if traffic_td:
    nearest_stations.append(traffic_td.get_text(strip=True))
else:
    station_spans = items.find_all("span", class_="prg-stationText")
    for span in station_spans:
        nearest_stations.append(span.get_text(strip=True))

# 各物件情報の表示
print("物件名称 (Property Name):", property_name)
print("住所 (Address):", address)
print("最寄り駅 (Nearest Stations):", nearest_stations)


物件名称 (Property Name): ＦＡＲＥ西新宿VIII
住所 (Address): 東京都新宿区西新宿4丁目20-14-2
最寄り駅 (Nearest Stations): ['都営大江戸線 西新宿五丁目駅 徒歩4分', '都営大江戸線 都庁前駅 徒歩12分']


↑ ひとまずHomes物件の名称、住所、最寄駅までスクレイピングできた。この後、築年数以降をスクレイピングで持ってきたい

スクレイピングできているかtxtファイルに出力して確認

In [4]:
# スクレイピングした HTML情報出力
f = open("test4.txt", "w")
f.write(str(items))
f.close()

築年数と何階建かは共通だが、それ以降は同じ建物でも部屋によって異なることに注意する！

In [5]:
module_body = items.find('div', class_='moduleBody')
if module_body:
        construction_th = module_body.find('th', string='築年数/階数')
        if construction_th:
            years_since_const = construction_th.find_next_sibling('td').get_text(strip=True).split(' ')[0]
            number_of_floors = construction_th.find_next_sibling('td').get_text(strip=True).split(' ')[2]  # 「新築」部分のみ取得
print("築年数 (Years Since Construction):", years_since_const)
print("階建 (Number of Floors):", number_of_floors)

築年数 (Years Since Construction): 新築
階建 (Number of Floors): 4階建


ここから同じ建物でも部屋によって異なる要素のスクレイピング

階数

In [6]:
floor_number_td   = items.find(class_="roomKaisuu")
floor_number      = floor_number_td.get_text(strip=True) if floor_number_td else None

print("階数 (Floor Number):", floor_number)

階数 (Floor Number): 4階


賃料と管理費

In [7]:
price_label = items.select_one("td.price .priceLabel")
print(price_label.get_text())

9.7万円


In [8]:
rent_price         = items.select_one("span.priceLabel")
rent_admin         = items.select_one("td.price .priceLabel").next_sibling.strip()

print(rent_admin)

/5,000円


つなぎ合わせて出力

In [9]:
rent_price_label = items.select_one("span.priceLabel")
rent_price = rent_price_label.get_text(strip=True) if rent_price_label else None

# 管理費の要素を選択
rent_admin = rent_price_label.next_sibling.strip().replace("/", "").replace(",", "") if rent_price_label else None

# 賃料と管理費が両方存在する場合に連結して出力
if rent_price and rent_admin:
    combined_rent = f"{rent_price} / {rent_admin}"
    print("賃料/管理費 (Rent/Administration Fee):", combined_rent)
else:
    print("賃料または管理費の要素が見つかりませんでした。")

賃料/管理費 (Rent/Administration Fee): 9.7万円 / 5000円


敷金、礼金も記載（スーモの形式　◯万円／◯万円　と合わせられていない）

In [10]:
price = items.select_one("td.price")
br_tag = price.find('br').next_sibling.strip()

depo = br_tag.split("/")[0]
key = br_tag.split("/")[1]

deposit_gratuity = f"{depo} / {key}"
print("賃料/管理費 (Rent/Administration Fee):", deposit_gratuity)

賃料/管理費 (Rent/Administration Fee): 1ヶ月 / 無


間取りと占有面積

In [11]:
layout = items.select_one("td.layout")
layout_text = layout.get_text(strip=True)

room_type = layout.contents[0].strip() if layout.contents else None
room_area = layout.find('br').next_sibling.strip() if layout.find('br') else None

layout_total_area = f"{room_type} / {room_area}"
print("間取り/占有面積 (Layout/Total Area):", layout_total_area)

間取り/占有面積 (Layout/Total Area): ワンルーム / 17.87m²


物件画像

In [12]:
property_image_element = items.select_one(".bukkenPhoto .photo img")
property_image_url = property_image_element["data-original"] if property_image_element else None

print("物件画像 URL (Property Image URL):", property_image_url)

物件画像 URL (Property Image URL): https://image4.homes.jp/smallimg/image.php?file=http%3A%2F%2Fimg.homes.jp%2F107095%2Frent%2F68049%2F2%2F2%2Fu1oj.jpg


間取り画像

In [13]:
floor_plan_image_element = items.select_one(".floarPlanPic img")
floor_plan_image_url = floor_plan_image_element["data-original"] if floor_plan_image_element else None
print("間取り情報画像 URL (Floor Plan Image URL):", floor_plan_image_url)

間取り情報画像 URL (Floor Plan Image URL): https://image3.homes.jp/smallimg/image.php?file=http%3A%2F%2Fimg.homes.jp%2F107095%2Frent%2F68049%2F1%2F1%2F6fvd.jpg


詳細URL

In [14]:
property_link_element = items.select_one("a[href*='/chintai/room']")
property_link =property_link_element['href'] if property_link_element else None 
## 不動産サイトから詳細URLリンクを読み解き作成
print("物件リンク (Property Link):", property_link)

物件リンク (Property Link): https://www.homes.co.jp/chintai/room/c36a2b7932339f6205ea77dad6a63af733483876/


全部くっつけてみる

In [15]:
base_url = "https://www.homes.co.jp/chintai/tokyo/shinjuku-city/list/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(base_url, headers=headers)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="mod-mergeBuilding--rent--photo")


#物件名称、住所、最寄駅
property_name = items.find(class_="bukkenName").get_text(strip=True) if items.find(class_="bukkenName") else None
    
for item in items:
    address_td = item.find('th', string='所在地')
    if address_td:
        address = address_td.find_next_sibling('td').get_text(strip=True)
        break

nearest_stations = []
traffic_td = items.find("td", class_="traffic")
if traffic_td:
    nearest_stations.append(traffic_td.get_text(strip=True))
else:
    station_spans = items.find_all("span", class_="prg-stationText")
    for span in station_spans:
        nearest_stations.append(span.get_text(strip=True))
# 各物件情報の表示
print("物件名称 (Property Name):", property_name)
print("住所 (Address):", address)
print("最寄り駅 (Nearest Stations):", nearest_stations)


#築年数、建物の階数
module_body = items.find('div', class_='moduleBody')
if module_body:
        construction_th = module_body.find('th', string='築年数/階数')
        if construction_th:
            years_since_const = construction_th.find_next_sibling('td').get_text(strip=True).split(' ')[0]
            number_of_floors = construction_th.find_next_sibling('td').get_text(strip=True).split(' ')[2]  # 「新築」部分のみ取得
print("築年数 (Years Since Construction):", years_since_const)
print("階建 (Number of Floors):", number_of_floors)


#当該部屋の階数
floor_number_td   = items.find(class_="roomKaisuu")
floor_number      = floor_number_td.get_text(strip=True) if floor_number_td else None
print("階数 (Floor Number):", floor_number)


#賃料と管理費
rent_price_label = items.select_one("span.priceLabel")
rent_price = rent_price_label.get_text(strip=True) if rent_price_label else None
# 管理費の要素を選択
rent_admin = rent_price_label.next_sibling.strip().replace("/", "").replace(",", "") if rent_price_label else None
# 賃料と管理費が両方存在する場合に連結して出力
if rent_price and rent_admin:
    combined_rent = f"{rent_price} / {rent_admin}"
    print("賃料/管理費 (Rent/Administration Fee):", combined_rent)
else:
    print("賃料または管理費の要素が見つかりませんでした。")


#賃料と管理費
price = items.select_one("td.price")
br_tag = price.find('br').next_sibling.strip()
depo = br_tag.split("/")[0]
key = br_tag.split("/")[1]
deposit_gratuity = f"{depo} / {key}"
print("賃料/管理費 (Rent/Administration Fee):", deposit_gratuity)


#間取りと占有面積
layout = items.select_one("td.layout")
layout_text = layout.get_text(strip=True)
room_type = layout.contents[0].strip() if layout.contents else None
room_area = layout.find('br').next_sibling.strip() if layout.find('br') else None
layout_total_area = f"{room_type} / {room_area}"
print("間取り/占有面積 (Layout/Total Area):", layout_total_area)


#物件画像
property_image_element = items.select_one(".bukkenPhoto .photo img")
property_image_url = property_image_element["data-original"] if property_image_element else None
print("物件画像 URL (Property Image URL):", property_image_url)


#間取り画像
floor_plan_image_element = items.select_one(".floarPlanPic img")
floor_plan_image_url = floor_plan_image_element["data-original"] if floor_plan_image_element else None
print("間取り情報画像 URL (Floor Plan Image URL):", floor_plan_image_url)


#物件詳細URL
property_link_element = items.select_one("a[href*='/chintai/room']")
property_link =property_link_element['href'] if property_link_element else None 
## 不動産サイトから詳細URLリンクを読み解き作成
print("物件リンク (Property Link):", property_link)

物件名称 (Property Name): ＦＡＲＥ西新宿VIII
住所 (Address): 東京都新宿区西新宿4丁目20-14-2
最寄り駅 (Nearest Stations): ['都営大江戸線 西新宿五丁目駅 徒歩4分', '都営大江戸線 都庁前駅 徒歩12分']
築年数 (Years Since Construction): 新築
階建 (Number of Floors): 4階建
階数 (Floor Number): 4階
賃料/管理費 (Rent/Administration Fee): 9.7万円 / 5000円
賃料/管理費 (Rent/Administration Fee): 1ヶ月 / 無
間取り/占有面積 (Layout/Total Area): ワンルーム / 17.87m²
物件画像 URL (Property Image URL): https://image4.homes.jp/smallimg/image.php?file=http%3A%2F%2Fimg.homes.jp%2F107095%2Frent%2F68049%2F2%2F2%2Fu1oj.jpg
間取り情報画像 URL (Floor Plan Image URL): https://image3.homes.jp/smallimg/image.php?file=http%3A%2F%2Fimg.homes.jp%2F107095%2Frent%2F68049%2F1%2F1%2F6fvd.jpg
物件リンク (Property Link): https://www.homes.co.jp/chintai/room/c36a2b7932339f6205ea77dad6a63af733483876/


複数のページから情報を取得する。

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.homes.co.jp/chintai/tokyo/shinjuku-city/list/?page={}"
max_page = 5
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url, headers=headers) 
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "mod-mergeBuilding--rent--photo"})

    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"] = item.find(class_="bukkenName").get_text(strip=True) if item.find(class_="bukkenName") else None
        base_data["アドレス"] = next((td.find_next_sibling('td').get_text(strip=True) for td in item.find_all('th') if td.get_text(strip=True) == '所在地'), None)
        base_data["アクセス"] = item.find("td", class_="traffic").get_text(strip=True) if item.find("td", class_="traffic") else ', '.join(span.get_text(strip=True) for span in item.find_all("span", class_="prg-stationText"))
        module_body = item.find('div', class_='moduleBody')
        if module_body:
            construction_th = module_body.find('th', string='築年数/階数')
            if construction_th:
                base_data["築年数"] = construction_th.find_next_sibling('td').get_text(strip=True).split(' ')[0]
                base_data["構造"] = construction_th.find_next_sibling('td').get_text(strip=True).split(' ')[2]

        rooms = item.find_all(class_="unitListBody prg-unitListBody")
        for room in rooms:
            data = base_data.copy()
            
            # 当該部屋の階数
            floor_number_td = room.find(class_="roomKaisuu")
            data["階数"] = floor_number_td.get_text(strip=True) if floor_number_td else None

            # 賃料と管理費を分けて取得
            rent_price_label = room.select_one("span.priceLabel")
            rent_price = rent_price_label.get_text(strip=True) if rent_price_label else None
            rent_admin = rent_price_label.next_sibling.strip().replace("/", "").replace(",", "") if rent_price_label else None
            data["家賃"] = rent_price
            data["管理費"] = rent_admin

            # 敷金と礼金を分けて取得
            price = room.select_one("td.price")
            br_tag = price.find('br').next_sibling.strip()
            depo = br_tag.split("/")[0]
            key = br_tag.split("/")[1]
            data["敷金"] = depo
            data["礼金"] = key

            # 間取りと占有面積を分けて取得
            layout = room.select_one("td.layout")
            room_type = layout.contents[0].strip() if layout.contents else None
            room_area = layout.find('br').next_sibling.strip().replace('m²', 'm2') if layout.find('br') else None
            data["間取り"] = room_type
            data["面積"] = room_area

            property_image_element = item.select_one(".bukkenPhoto .photo img")
            data["物件画像URL"] = property_image_element["data-original"] if property_image_element else None

            # 間取り画像URL
            floor_plan_image_element = item.select_one(".floarPlanPic img")
            data["間取画像URL"] = floor_plan_image_element["data-original"] if floor_plan_image_element else None

            # 物件詳細URL
            property_link_element = item.select_one("a[href*='/chintai/room']")
            data["物件詳細URL"] = property_link_element['href'] if property_link_element else None


            all_data.append(data)

page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30


In [17]:
df = pd.DataFrame(all_data)
df.head(5)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,ＦＡＲＥ西新宿VIII,東京都新宿区西新宿4丁目20-14-2,"都営大江戸線 西新宿五丁目駅 徒歩4分, 都営大江戸線 都庁前駅 徒歩12分",新築,4階建,4階,9.7万円,5000円,1ヶ月,無,ワンルーム,17.87m2,https://image4.homes.jp/smallimg/image.php?fil...,https://image3.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/c36a2b793...
1,ＨＪ ＰＬＡＣＥ 早稲田鶴巻 I,東京都新宿区早稲田鶴巻町107-20,"東京メトロ東西線 早稲田駅 徒歩6分, 東京メトロ有楽町線 江戸川橋駅 徒歩11分",新築,5階建,5階,7.9万円,8000円,無,無,ワンルーム,12.16m2,https://image3.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/d06fc37e0...
2,オアーゼ新宿市谷薬王寺,東京都新宿区市谷薬王寺町80-1,"都営大江戸線 牛込柳町駅 徒歩6分, 都営新宿線 曙橋駅 徒歩7分",2年,6階建,2階,24万円,10000円,1ヶ月,1ヶ月,1SLDK,49.15m2,https://image2.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/dcd431980...
3,PASEO市谷加賀町,東京都新宿区市谷加賀町2丁目,"都営大江戸線 牛込柳町駅 徒歩10分, 都営新宿線 曙橋駅 徒歩11分, JR中央線 市ケ谷...",2年,4階建,3階,11.9万円,10000円,無,無,1K,23.4m2,https://image2.homes.jp/smallimg/image.php?fil...,https://image3.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/6adc31d49...
4,レピュア神楽坂駅前レジデンス,東京都新宿区細工町1-15,"都営大江戸線 牛込神楽坂駅 徒歩3分, 東京メトロ東西線 神楽坂駅 徒歩8分",新築,5階建,3階,13.8万円,12000円,無,無,ワンルーム,20.31m2,https://image2.homes.jp/smallimg/image.php?fil...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/eca767a0d...


スーモと重複削除するために住所の表記を合わせる

In [18]:
# 住所の標準化関数
def standardize_address(address):
    # 数字を半角から全角に変換
    address = re.sub(r'1', '１', address)
    address = re.sub(r'2', '２', address)
    address = re.sub(r'3', '３', address)
    address = re.sub(r'4', '４', address)
    address = re.sub(r'5', '５', address)
    address = re.sub(r'6', '６', address)
    address = re.sub(r'7', '７', address)
    address = re.sub(r'8', '８', address)
    address = re.sub(r'9', '９', address)
    address = re.sub(r'0', '０', address)
    
    # 「丁目」以降を削除
    address = re.sub(r'丁目.*', '', address)
    address = re.sub(r'-.*', '', address)
    
    return address

# 標準化した住所を新しいカラムに追加
df['アドレス'] = df['アドレス'].apply(standardize_address)

#数字を除去した住所をアドレス_数字除去に格納
def remove_numbers(address):
    # 数字を削除
    address_no_numbers = re.sub(r'[0-9０-９]', '', address)
    return address_no_numbers

# 数字を削除した住所を新しいカラムに追加
df['アドレス_数字除去'] = df['アドレス'].apply(remove_numbers)

In [19]:
df.head(5)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,アドレス_数字除去
0,ＦＡＲＥ西新宿VIII,東京都新宿区西新宿４,"都営大江戸線 西新宿五丁目駅 徒歩4分, 都営大江戸線 都庁前駅 徒歩12分",新築,4階建,4階,9.7万円,5000円,1ヶ月,無,ワンルーム,17.87m2,https://image4.homes.jp/smallimg/image.php?fil...,https://image3.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/c36a2b793...,東京都新宿区西新宿
1,ＨＪ ＰＬＡＣＥ 早稲田鶴巻 I,東京都新宿区早稲田鶴巻町１０７,"東京メトロ東西線 早稲田駅 徒歩6分, 東京メトロ有楽町線 江戸川橋駅 徒歩11分",新築,5階建,5階,7.9万円,8000円,無,無,ワンルーム,12.16m2,https://image3.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/d06fc37e0...,東京都新宿区早稲田鶴巻町
2,オアーゼ新宿市谷薬王寺,東京都新宿区市谷薬王寺町８０,"都営大江戸線 牛込柳町駅 徒歩6分, 都営新宿線 曙橋駅 徒歩7分",2年,6階建,2階,24万円,10000円,1ヶ月,1ヶ月,1SLDK,49.15m2,https://image2.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/dcd431980...,東京都新宿区市谷薬王寺町
3,PASEO市谷加賀町,東京都新宿区市谷加賀町２,"都営大江戸線 牛込柳町駅 徒歩10分, 都営新宿線 曙橋駅 徒歩11分, JR中央線 市ケ谷...",2年,4階建,3階,11.9万円,10000円,無,無,1K,23.4m2,https://image2.homes.jp/smallimg/image.php?fil...,https://image3.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/6adc31d49...,東京都新宿区市谷加賀町
4,レピュア神楽坂駅前レジデンス,東京都新宿区細工町１,"都営大江戸線 牛込神楽坂駅 徒歩3分, 東京メトロ東西線 神楽坂駅 徒歩8分",新築,5階建,3階,13.8万円,12000円,無,無,ワンルーム,20.31m2,https://image2.homes.jp/smallimg/image.php?fil...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/eca767a0d...,東京都新宿区細工町


ここで、スーモからも出力！

In [20]:
# 基本URLと最大ページ数の設定
base_url = "https://suumo.jp/chintai/tokyo/sc_shinjuku/?page={}"
max_page = 5  # 最大ページ数

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "cassetteitem"})


    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"]     = item.find("div", {"class": "cassetteitem_content-title"}).get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-title"}) else None
        base_data["カテゴリ"] = item.find("div", {"class": "cassetteitem_content-label"}).span.get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-label"}) else None
        base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).get_text(strip=True) if item.find("li", {"class": "cassetteitem_detail-col1"}) else None
        
        # 駅のアクセス情報をまとめて取得
        base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

        construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
        base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
        base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

        for tbody in tbodys:
            data = base_data.copy()
            # 階数情報の正確な取得
            floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
            data["階数"]   = floor_info
            data["家賃"]   = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
            data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
            data["敷金"]   = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
            data["礼金"]   = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
            data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
            data["面積"]   = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

            # 物件画像・間取り画像・詳細URLの取得を最後に行う
            property_image_element = item.find(class_="cassetteitem_object-item")
            data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

            floor_plan_image_element = item.find(class_="casssetteitem_other-thumbnail")
            data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

            property_link_element = item.select_one("a[href*='/chintai/jnc_']")
            data["物件詳細URL"] = "https://suumo.jp" +property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

            all_data.append(data)    


page 1 items 20
page 2 items 20
page 3 items 20
page 4 items 20
page 5 items 20


スーモのデータはdf2へ

In [21]:
df2 = pd.DataFrame(all_data)

In [22]:
df2.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,Ｍｏｎｋｓｔｏｗｎ夏目坂,賃貸マンション,東京都新宿区戸山１,"東京メトロ東西線/早稲田駅 歩4分, 都営大江戸線/若松河田駅 歩10分, 都営大江戸線/牛...",新築,4階建,1階,28.5万円,10000円,28.5万円,28.5万円,1SLDK,75.09m2,https://img01.suumo.com/front/gazo/fr/bukken/3...,https://img01.suumo.com/front/gazo/fr/bukken/3...,https://suumo.jp/chintai/jnc_000089432312/?bc=...
1,プライムアーバン新宿夏目坂タワーレジデンス,賃貸マンション,東京都新宿区原町３,"都営大江戸線/牛込柳町駅 歩4分, 都営大江戸線/若松河田駅 歩5分, 東京メトロ東西線/早...",築17年,地下1地上30階建,2階,18.8万円,10000円,37.6万円,-,1LDK,43.86m2,https://img01.suumo.com/front/gazo/fr/bukken/8...,https://img01.suumo.com/front/gazo/fr/bukken/7...,https://suumo.jp/chintai/jnc_000067761648/?bc=...


In [23]:
#数字を除去した住所をアドレス_数字除去に格納
def remove_numbers(address):
    # 数字を削除
    address_no_numbers = re.sub(r'[0-9０-９]', '', address)
    return address_no_numbers

# 数字を削除した住所を新しいカラムに追加
df2['アドレス_数字除去'] = df2['アドレス'].apply(remove_numbers)

カテゴリはホームズにないので消す

In [24]:
df2_cleaned = df2.drop(columns=['カテゴリ'])

In [25]:
df2_cleaned.head(1)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,アドレス_数字除去
0,Ｍｏｎｋｓｔｏｗｎ夏目坂,東京都新宿区戸山１,"東京メトロ東西線/早稲田駅 歩4分, 都営大江戸線/若松河田駅 歩10分, 都営大江戸線/牛...",新築,4階建,1階,28.5万円,10000円,28.5万円,28.5万円,1SLDK,75.09m2,https://img01.suumo.com/front/gazo/fr/bukken/3...,https://img01.suumo.com/front/gazo/fr/bukken/3...,https://suumo.jp/chintai/jnc_000089432312/?bc=...,東京都新宿区戸山


dfとdf2_cleanを合体させる

In [26]:
df_merged = pd.concat([df, df2_cleaned], ignore_index=True)

In [27]:
df_merged.head(1)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,アドレス_数字除去
0,ＦＡＲＥ西新宿VIII,東京都新宿区西新宿４,"都営大江戸線 西新宿五丁目駅 徒歩4分, 都営大江戸線 都庁前駅 徒歩12分",新築,4階建,4階,9.7万円,5000円,1ヶ月,無,ワンルーム,17.87m2,https://image4.homes.jp/smallimg/image.php?fil...,https://image3.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/c36a2b793...,東京都新宿区西新宿


重複物件を削除する。削除の条件は、「築年数」「構造」「階数」「家賃」「面積」「アドレス_数字除去」カラムが一致していること。

In [28]:
# 重複削除
df_deduplicated = df_merged.drop_duplicates(subset=['築年数', '構造', '階数', '家賃', '面積', 'アドレス_数字除去'])

# 削除前後の行数を比較
original_count = len(df_merged)
deduplicated_count = len(df_deduplicated)
duplicates_removed = original_count - deduplicated_count

# 結果の表示
print(f"\n削除された重複レコード数: {duplicates_removed}")


削除された重複レコード数: 88


df_deduplicatedから「アドレス_数字除去」を削除する

In [29]:
df_deduplicated = df_deduplicated.drop(columns=['アドレス_数字除去'])
df_deduplicated.head(1)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,ＦＡＲＥ西新宿VIII,東京都新宿区西新宿４,"都営大江戸線 西新宿五丁目駅 徒歩4分, 都営大江戸線 都庁前駅 徒歩12分",新築,4階建,4階,9.7万円,5000円,1ヶ月,無,ワンルーム,17.87m2,https://image4.homes.jp/smallimg/image.php?fil...,https://image3.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/c36a2b793...


In [30]:
df_deduplicated.tail(1)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
732,都営大江戸線 牛込柳町駅 7階建 築22年,東京都新宿区若松町,"都営大江戸線/牛込柳町駅 歩3分, 都営新宿線/曙橋駅 歩15分, 東京メトロ東西線/早稲田...",築22年,7階建,4階,9万円,5000円,9万円,9万円,1K,26.4m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://suumo.jp/chintai/jnc_000041794949/?bc=...


ここからSQLiteに格納する処理！

In [31]:
import sqlite3

SQLiteにデータを格納

In [33]:
conn = sqlite3.connect('chintai.db')

# データフレームをSQLiteに格納
df_deduplicated.to_sql('properties', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()

SQLiteからデータを読み込んでこちらで加工

In [34]:
# SQLiteデータベースへの接続
conn = sqlite3.connect('test_20240521.db')

# データベースからデータを読み込む
df = pd.read_sql_query('SELECT * FROM properties', conn)

不動産データを加工する

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   名称       645 non-null    object
 1   アドレス     645 non-null    object
 2   アクセス     645 non-null    object
 3   築年数      645 non-null    object
 4   構造       645 non-null    object
 5   階数       645 non-null    object
 6   家賃       645 non-null    object
 7   管理費      645 non-null    object
 8   敷金       645 non-null    object
 9   礼金       645 non-null    object
 10  間取り      645 non-null    object
 11  面積       645 non-null    object
 12  物件画像URL  645 non-null    object
 13  間取画像URL  645 non-null    object
 14  物件詳細URL  645 non-null    object
dtypes: object(15)
memory usage: 75.7+ KB


新築であれば築年数を0にする。また、次のように加工：「築10年」 → 10、「10年」→ 10

In [36]:
# 数字を抽出する関数
def extract_years(x):
    if x == '新築':
        return 0
    match = re.search(r'(\d+)', x)
    if match:
        return int(match.group(1))
    return None  # 予期しない値の場合

# 築年数を変換
df['築年数'] = df["築年数"].apply(extract_years)

構造を変更。「階建」を消す。

In [37]:
def get_most_floor(x):
    if '階建' not in x:
        return np.nan
    else:
        list = re.findall(r'(\d+)階建', str(x))
        if list:
            list = map(int, list)
            min_value = min(list)
            return min_value
        return np.nan

# 適用
df['構造'] = df['構造'].apply(get_most_floor)

階数を編集する。「5階」→5

In [38]:
# 階数を抽出する関数
def get_floor(x):
    if isinstance(x, float) and np.isnan(x):
        return 0  # NaNの場合は0を返す
    x = str(x)  # 文字列に変換
    if '階' not in x:
        return 0  # 階数がない場合は0を返す
    elif 'B' not in x:
        floors = re.findall(r'(\d+)階', x)
        floors = list(map(int, floors))
        min_value = min(floors)
        return int(min_value)  # 結果を整数型で返す
    else:
        floors = re.findall(r'(\d+)階', x)
        floors = list(map(int, floors))
        min_value = -1 * min(floors)
        return int(min_value)  # 結果を整数型で返す

# 適用
df['階数'] = df['階数'].apply(get_floor)


In [39]:
print(df['階数'].head(5))

0    4
1    5
2    2
3    3
4    3
Name: 階数, dtype: int64


家賃を数字に変える

In [40]:
def change_fee(x):
    if ('万円' not in x) :
        return np.nan
    else:
        return float(x.split('万円')[0])

df['家賃'] = df['家賃'].apply(change_fee)

敷金、礼金が「１ヶ月」や「無」の場合に数字に変換して、さらに０以外の場合は家賃との掛け算をおこなう

In [41]:
# 敷金・礼金の変換関数
def convert_fee(value):
    if isinstance(value, float) and pd.isna(value):
        return 0  # NaNの場合は0を返す
    value = str(value)  # 文字列に変換
    if '無' in value:
        return 0
    match = re.match(r'(\d+)ヶ月', value)
    if match:
        return int(match.group(1))
    return 0  # 予期しない値の場合は0を返す

# 敷金と礼金のカラムに変換関数を適用
df['敷金'] = df['敷金'].apply(convert_fee)
df['礼金'] = df['礼金'].apply(convert_fee)

敷金、礼金は現状「○ヶ月分」を表すので家賃と掛け算しておく。これで○万円になる

In [42]:
df['敷金'] = df['敷金'] * df['家賃']
df['礼金'] = df['礼金'] * df['家賃']

In [43]:
def change_fee2(x):
    if ('円' not in x) :
        return np.nan
    else:
        return float(x.split('円')[0])


df['管理費'] = df['管理費'].apply(change_fee2)

In [44]:
df['面積'] = df['面積'].apply(lambda x: float(x[:-2]))

In [45]:
df['区'] = df["アドレス"].apply(lambda x : x[x.find("都")+1:x.find("区")+1])

In [46]:
df['市町'] = df["アドレス"].apply(lambda x : x[x.find("区")+1 :-1])

「西武新宿線/鷺ノ宮駅 歩9分」「西武新宿線 鷺ノ宮駅 徒歩9分」どちらの表記であっても上手く分割できるように書き換える

In [47]:

def split_access(row):
    accesses = row['アクセス'].split(', ')
    results = {}

    for i, access in enumerate(accesses, start=1):
        if i > 3:
            break  # 最大3つのアクセス情報のみを考慮

        parts = re.split(r'[ /]', access, maxsplit=1)
        if len(parts) == 2:
            line_station, walk = parts
            line_station = line_station.strip()
            walk = walk.strip()

            # '徒歩'または'歩'で分割できるか確認
            walk_min_match = re.search(r'(徒歩|歩)(\d+)分', walk)
            if walk_min_match:
                station = re.split(r'(駅)', line_station, maxsplit=1)[0].strip() + '駅'
                walk_min = int(walk_min_match.group(2))
            else:
                station = None
                walk_min = None
        else:
            line_station = access.strip()
            station = walk_min = None

        results[f'アクセス{i}線路名'] = line_station
        results[f'アクセス{i}駅名'] = station
        results[f'アクセス{i}徒歩(分)'] = walk_min

    return pd.Series(results)

# 新しい列をデータフレームに適用
df = df.join(df.apply(split_access, axis=1))

In [48]:
df.head(2)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,...,市町,アクセス1徒歩(分),アクセス1線路名,アクセス1駅名,アクセス2徒歩(分),アクセス2線路名,アクセス2駅名,アクセス3徒歩(分),アクセス3線路名,アクセス3駅名
0,ＦＡＲＥ西新宿VIII,東京都新宿区西新宿４,"都営大江戸線 西新宿五丁目駅 徒歩4分, 都営大江戸線 都庁前駅 徒歩12分",0,4,4,9.7,5000.0,9.7,0.0,...,西新宿,4.0,都営大江戸線,都営大江戸線駅,12.0,都営大江戸線,都営大江戸線駅,NaN,NaN,NaN
1,ＨＪ ＰＬＡＣＥ 早稲田鶴巻 I,東京都新宿区早稲田鶴巻町１０７,"東京メトロ東西線 早稲田駅 徒歩6分, 東京メトロ有楽町線 江戸川橋駅 徒歩11分",0,5,5,7.9,8000.0,0.0,0.0,...,早稲田鶴巻町１０,6.0,東京メトロ東西線,東京メトロ東西線駅,11.0,東京メトロ有楽町線,東京メトロ有楽町線駅,NaN,NaN,NaN


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   名称          645 non-null    object 
 1   アドレス        645 non-null    object 
 2   アクセス        645 non-null    object 
 3   築年数         645 non-null    int64  
 4   構造          645 non-null    int64  
 5   階数          645 non-null    int64  
 6   家賃          645 non-null    float64
 7   管理費         591 non-null    float64
 8   敷金          645 non-null    float64
 9   礼金          645 non-null    float64
 10  間取り         645 non-null    object 
 11  面積          645 non-null    float64
 12  物件画像URL     645 non-null    object 
 13  間取画像URL     645 non-null    object 
 14  物件詳細URL     645 non-null    object 
 15  区           645 non-null    object 
 16  市町          645 non-null    object 
 17  アクセス1徒歩(分)  642 non-null    float64
 18  アクセス1線路名    645 non-null    object 
 19  アクセス1駅名     642 non-null    o

In [50]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import pandas as pd
import time


# ジオコーダーの初期化
geolocator = Nominatim(user_agent="your_app_name", timeout=10)

current_count = 0
total_count = len(df['アドレス'])

# 住所から緯度と経度を取得する関数
def get_lat_lon(address, retries=3):
    global current_count
    current_count += 1
    
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                print(f"{current_count}/{total_count} 件目実施中 結果: {location.latitude}, {location.longitude}")
                return location.latitude, location.longitude
            else:
                print(f"{current_count}/{total_count} 件目実施中 結果: 住所が見つかりません")
                return None, None
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error retrieving location for address {address}: {e}. Retrying ({attempt + 1}/{retries})...")
            time.sleep(1)  # リトライの前に1秒待つ

    print(f"Failed to retrieve location for address {address} after {retries} retries")
    return None, None

# 新しい列を作成してデータフレームに追加
df['緯度'], df['経度'] = zip(*df['アドレス'].apply(get_lat_lon))

# 結果の表示
print(df.head())

1/645 件目実施中 結果: 35.6870387, 139.6883807
2/645 件目実施中 結果: 住所が見つかりません
3/645 件目実施中 結果: 住所が見つかりません
4/645 件目実施中 結果: 住所が見つかりません
5/645 件目実施中 結果: 住所が見つかりません
6/645 件目実施中 結果: 住所が見つかりません
7/645 件目実施中 結果: 住所が見つかりません
8/645 件目実施中 結果: 35.7067623, 139.7282444200764
9/645 件目実施中 結果: 住所が見つかりません
10/645 件目実施中 結果: 住所が見つかりません
11/645 件目実施中 結果: 35.7082189, 139.7122677
12/645 件目実施中 結果: 住所が見つかりません
13/645 件目実施中 結果: 35.70234595, 139.69479256063295
14/645 件目実施中 結果: 住所が見つかりません
15/645 件目実施中 結果: 住所が見つかりません
16/645 件目実施中 結果: 35.7075368, 139.7367933317518
17/645 件目実施中 結果: 住所が見つかりません
18/645 件目実施中 結果: 35.70818, 139.6938483860959
19/645 件目実施中 結果: 住所が見つかりません
20/645 件目実施中 結果: 住所が見つかりません
21/645 件目実施中 結果: 住所が見つかりません
22/645 件目実施中 結果: 35.721371, 139.6799967
23/645 件目実施中 結果: 35.7082189, 139.7122677
24/645 件目実施中 結果: 住所が見つかりません
25/645 件目実施中 結果: 住所が見つかりません
26/645 件目実施中 結果: 35.70234595, 139.69479256063295
27/645 件目実施中 結果: 35.70234595, 139.69479256063295
28/645 件目実施中 結果: 35.7075368, 139.7367933317518
29/645 件目実施中 結果: 住所が見つかりません
30/645 件目

In [51]:
conn = sqlite3.connect('chintai.db')

# データフレームをSQLiteに格納
df.to_sql('properties', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()